# Data Validation: Fills vs Summary
Validate consistency between fills table and account_daily_summary table

In [ ]:
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Database connection
DB_PATH = Path("../data/trading_risk.db")
conn = sqlite3.connect(DB_PATH)

# Load data
fills = pd.read_sql_query("SELECT * FROM fills", conn)
summaries = pd.read_sql_query("SELECT * FROM account_daily_summary", conn)

print(f"Fills shape: {fills.shape}")
print(f"Summaries shape: {summaries.shape}")

In [ ]:
# Prepare fills data for validation
fills['datetime'] = pd.to_datetime(fills['datetime'])
fills['date'] = fills['datetime'].dt.date
fills['date_str'] = fills['date'].astype(str)

# Prepare summaries data
summaries['date'] = pd.to_datetime(summaries['date']).dt.date
summaries['date_str'] = summaries['date'].astype(str)

print("Date ranges:")
print(f"Fills: {fills['date'].min()} to {fills['date'].max()}")
print(f"Summaries: {summaries['date'].min()} to {summaries['date'].max()}")

In [ ]:
# 1. VALIDATE DAILY FILL COUNTS
fills_daily_count = fills.groupby(['account_id', 'date_str']).size().reset_index(name='fills_count')
validation_df = summaries.merge(
    fills_daily_count, 
    on=['account_id', 'date_str'], 
    how='outer', 
    indicator=True
)

# Check for mismatches
validation_df['fills_match'] = validation_df['fills'] == validation_df['fills_count']

print("Fill count validation:")
print(f"Total records: {len(validation_df)}")
print(f"Matching fill counts: {validation_df['fills_match'].sum()}")
print(f"Mismatches: {(~validation_df['fills_match']).sum()}")
print(f"\nMerge indicator counts:")
print(validation_df['_merge'].value_counts())

In [ ]:
# Show mismatches
mismatches = validation_df[~validation_df['fills_match'] & (validation_df['_merge'] == 'both')]
if len(mismatches) > 0:
    print(f"\nSample of fill count mismatches:")
    display(mismatches[['account_id', 'date_str', 'fills', 'fills_count']].head(10))

In [ ]:
# 2. VALIDATE DAILY QUANTITIES
fills_daily_qty = fills.groupby(['account_id', 'date_str'])['quantity'].sum().reset_index(name='fills_qty')
qty_validation = summaries.merge(
    fills_daily_qty, 
    on=['account_id', 'date_str'], 
    how='inner'
)

qty_validation['qty_match'] = np.isclose(qty_validation['qty'], qty_validation['fills_qty'], rtol=0.01)
qty_validation['qty_diff'] = qty_validation['qty'] - qty_validation['fills_qty']
qty_validation['qty_diff_pct'] = (qty_validation['qty_diff'] / qty_validation['qty'] * 100).round(2)

print("Quantity validation:")
print(f"Matching quantities: {qty_validation['qty_match'].sum()} / {len(qty_validation)}")
print(f"Average difference: {qty_validation['qty_diff'].mean():.2f}")
print(f"Max absolute difference: {qty_validation['qty_diff'].abs().max():.2f}")

In [ ]:
# 3. VALIDATE GROSS PNL
# Calculate gross from fills (buy negative, sell positive)
fills['signed_value'] = fills.apply(
    lambda x: -x['price'] * x['quantity'] if x['side'] == 'B' else x['price'] * x['quantity'], 
    axis=1
)

fills_daily_gross = fills.groupby(['account_id', 'date_str'])['signed_value'].sum().reset_index(name='fills_gross')
gross_validation = summaries.merge(
    fills_daily_gross, 
    on=['account_id', 'date_str'], 
    how='inner'
)

gross_validation['gross_match'] = np.isclose(gross_validation['gross'], gross_validation['fills_gross'], rtol=0.01)
gross_validation['gross_diff'] = gross_validation['gross'] - gross_validation['fills_gross']
gross_validation['gross_diff_pct'] = (gross_validation['gross_diff'] / gross_validation['gross'].abs() * 100).replace([np.inf, -np.inf], 0).round(2)

print("Gross PnL validation:")
print(f"Matching gross: {gross_validation['gross_match'].sum()} / {len(gross_validation)}")
print(f"Average difference: ${gross_validation['gross_diff'].mean():.2f}")
print(f"Max absolute difference: ${gross_validation['gross_diff'].abs().max():.2f}")

In [ ]:
# 4. VALIDATE FEES
# Sum all fee columns from fills
fee_columns = ['commission', 'ecn_fee', 'sec_fee', 'orf_fee', 'cat_fee', 
               'taf_fee', 'ftt_fee', 'nscc_fee', 'acc_fee', 'clr_fee', 'misc_fee']

fills_daily_fees = fills.groupby(['account_id', 'date_str'])[fee_columns].sum().reset_index()
fills_daily_fees['fills_total_fees'] = fills_daily_fees[fee_columns].sum(axis=1)

# Merge with summaries
fee_validation = summaries.merge(
    fills_daily_fees[['account_id', 'date_str', 'fills_total_fees']], 
    on=['account_id', 'date_str'], 
    how='inner'
)

fee_validation['fee_match'] = np.isclose(fee_validation['trade_fees'], fee_validation['fills_total_fees'], rtol=0.01)
fee_validation['fee_diff'] = fee_validation['trade_fees'] - fee_validation['fills_total_fees']

print("Fee validation:")
print(f"Matching fees: {fee_validation['fee_match'].sum()} / {len(fee_validation)}")
print(f"Average difference: ${fee_validation['fee_diff'].mean():.2f}")
print(f"Max absolute difference: ${fee_validation['fee_diff'].abs().max():.2f}")

In [ ]:
# 5. VALIDATION SUMMARY VISUALIZATION
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Data Validation: Fills vs Summary', fontsize=16)

# 5.1 Fill count differences
ax1 = axes[0, 0]
count_diff = validation_df[validation_df['_merge'] == 'both']['fills'] - validation_df[validation_df['_merge'] == 'both']['fills_count']
ax1.hist(count_diff.dropna(), bins=50, alpha=0.7, color='blue')
ax1.axvline(0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('Fill Count Difference (Summary - Fills)')
ax1.set_ylabel('Frequency')
ax1.set_title('Fill Count Discrepancies')

# 5.2 Quantity differences
ax2 = axes[0, 1]
ax2.scatter(qty_validation['qty'], qty_validation['qty_diff_pct'], alpha=0.5)
ax2.axhline(0, color='red', linestyle='--')
ax2.set_xlabel('Summary Quantity')
ax2.set_ylabel('Difference (%)')
ax2.set_title('Quantity Validation')

# 5.3 Gross PnL differences  
ax3 = axes[1, 0]
ax3.scatter(gross_validation['gross'], gross_validation['gross_diff'], alpha=0.5)
ax3.axhline(0, color='red', linestyle='--')
ax3.set_xlabel('Summary Gross PnL')
ax3.set_ylabel('Difference ($)')
ax3.set_title('Gross PnL Validation')

# 5.4 Fee differences
ax4 = axes[1, 1]
ax4.scatter(fee_validation['trade_fees'], fee_validation['fee_diff'], alpha=0.5)
ax4.axhline(0, color='red', linestyle='--')
ax4.set_xlabel('Summary Fees')
ax4.set_ylabel('Difference ($)')
ax4.set_title('Fee Validation')

plt.tight_layout()
plt.show()

In [ ]:
# 6. IDENTIFY PROBLEMATIC RECORDS
# Combine all validation flags
final_validation = summaries.copy()
final_validation = final_validation.merge(
    validation_df[['account_id', 'date_str', 'fills_match']], 
    on=['account_id', 'date_str'], 
    how='left'
)
final_validation = final_validation.merge(
    qty_validation[['account_id', 'date_str', 'qty_match']], 
    on=['account_id', 'date_str'], 
    how='left'
)
final_validation = final_validation.merge(
    gross_validation[['account_id', 'date_str', 'gross_match']], 
    on=['account_id', 'date_str'], 
    how='left'
)

# Overall validation flag
final_validation['all_valid'] = (
    final_validation['fills_match'].fillna(False) & 
    final_validation['qty_match'].fillna(False) & 
    final_validation['gross_match'].fillna(False)
)

print("\n=== VALIDATION SUMMARY ===")
print(f"Total summary records: {len(final_validation)}")
print(f"Fully valid records: {final_validation['all_valid'].sum()} ({final_validation['all_valid'].sum()/len(final_validation)*100:.1f}%)")
print(f"\nValidation by component:")
print(f"- Fill counts valid: {final_validation['fills_match'].sum()} ({final_validation['fills_match'].sum()/len(final_validation)*100:.1f}%)")
print(f"- Quantities valid: {final_validation['qty_match'].sum()} ({final_validation['qty_match'].sum()/len(final_validation)*100:.1f}%)")
print(f"- Gross PnL valid: {final_validation['gross_match'].sum()} ({final_validation['gross_match'].sum()/len(final_validation)*100:.1f}%)")

# Show problematic accounts
problem_accounts = final_validation[~final_validation['all_valid']]['account_id'].value_counts()
print(f"\nAccounts with validation issues:")
display(problem_accounts.head(10))

In [ ]:
# 7. SAVE VALIDATION RESULTS
validation_summary = {
    'total_summary_records': len(summaries),
    'total_fills_records': len(fills),
    'valid_records': final_validation['all_valid'].sum(),
    'invalid_records': (~final_validation['all_valid']).sum(),
    'validation_rate': final_validation['all_valid'].sum() / len(final_validation),
    'accounts_with_issues': len(problem_accounts),
    'date_range_match': fills['date'].min() == summaries['date'].min() and fills['date'].max() == summaries['date'].max()
}

print("\n📊 FINAL VALIDATION REPORT:")
for key, value in validation_summary.items():
    print(f"{key}: {value}")

# Export problematic records for investigation
if (~final_validation['all_valid']).sum() > 0:
    problematic = final_validation[~final_validation['all_valid']]
    print(f"\n⚠️  Found {len(problematic)} records with validation issues")
    print("Sample of problematic records:")
    display(problematic[['account_id', 'date_str', 'fills_match', 'qty_match', 'gross_match']].head())